## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [3]:
import pandas as pd
sсore_data = pd.read_csv('/datasets/data.csv')
print(sсore_data.info())

sсore_data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Для анализа данных (обработки) представлен файл, содержащий таблицу из 21525 строк, 12 колонок. 
Типы данных в колонках:5 колонок с текстовыми данными, 5 колонок с целочисленными выражениями, 2 колонки с вещественными (дробными числами).
С первого взгляда налюдается дублирование данных в колонке education. 


### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
sсore_data['children'].value_counts() #для каждого столбца проверям наличие пропущенных значений
#вызывает сомнения корректность данных по категории 20 детей, отрицательное значение. 
# 47 строк с отрицтельным значением и 76 со значением, вызывающим сомнения. Будем считать эти данные флуктуацией.
# в условиях проекта 123 строки составляют менее 1% от всего объема данных и могут быть удалены.
# в реальной жизни необходимо оценивать требования по задаче, сроки и ресурсы на возможную коммуникацию с заказчиком по исправлени данных.
# в любом случае, необходимо проифнормировать об ошибке в данных или сделать пометку в пояснительной записке к отчету.



 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [5]:
sсore_data[sсore_data['children'].isnull()].count() #для каждого столбца проверяем наличие пропущенных строк

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [6]:
sсore_data = sсore_data[(sсore_data['children'] != 20) & (sсore_data['children'] != -1)]
sсore_data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [7]:
sсore_data['days_employed'].value_counts()#видим отрицательные значения в колонке с общим трудовым стажем в днях
# в рамках текущего проекта колонка не является первичной для изучения и анализа, поэтому, допускаю оставить без изменений.
# в реальной жизни неоходимо либо обратиться в к заказчику, либо сделать допущение, что это техническая ошибка 
#и заменить все отрицательные данные на положительные (более 15 тыс. строк)


-986.927316       1
-849.764227       1
-5135.928528      1
-1453.358707      1
-4977.646061      1
                 ..
-2348.524271      1
-2338.480708      1
 356642.853685    1
-1218.820922      1
-582.538413       1
Name: days_employed, Length: 19240, dtype: int64

In [8]:
print(sсore_data['days_employed'][sсore_data['days_employed'] < 0].count())

15809


In [9]:
sсore_data[sсore_data['days_employed'].isnull()].count() #в столбце общий трудовой стаж в днях видим 2152 строки с пропущенным значением. 
# возможные причины - данные отсутствуют, ошибка выгрузки.
#заполним их

children            2162
days_employed          0
dob_years           2162
education           2162
education_id        2162
family_status       2162
family_status_id    2162
gender              2162
income_type         2162
debt                2162
total_income           0
purpose             2162
dtype: int64

In [10]:
sсore_data['update_days_employed'] = sсore_data['days_employed'] 
#вводим новый столбец для заполнения медианным показателем стажа 
#(в нем в цикле будем заполнять пропуски медианными значениями)
for income_type in sсore_data['income_type'].unique(): 
    #находим уникальные значение в категории тип занятости 
    #(медианный стаж ищем для каждого типа занятости)
    med_days_employed = sсore_data[sсore_data['income_type'] == income_type]['days_employed'].median()
    sсore_data.loc[(sсore_data['update_days_employed'].isna()) & (sсore_data['income_type'] == income_type), 'update_days_employed'] = med_days_employed
    
    

In [11]:
sсore_data[sсore_data['update_days_employed'].isnull()].count()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [12]:
sсore_data['dob_years'].value_counts().sort_values(ascending = True) #вызывает сомнения возможность выдачи кредитов группе 
#с возрастом 0
# 100 строк составляют менее 0.5% от всего объема данных и могут быть удалены.

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     83
68     99
0     100
21    110
67    167
22    183
66    183
65    194
23    252
24    263
64    263
63    268
62    351
61    353
25    356
60    376
26    406
59    441
55    441
51    446
53    457
57    457
58    461
46    469
54    476
47    480
56    482
52    483
27    490
45    494
28    501
49    505
32    506
50    509
43    510
37    531
48    536
30    536
44    543
29    543
36    553
31    556
39    572
33    577
42    592
38    595
34    597
41    603
40    603
35    614
Name: dob_years, dtype: int64

In [13]:
sсore_data = sсore_data[sсore_data['dob_years'] != 0]
sсore_data['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [14]:
sсore_data[sсore_data['dob_years'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [15]:
sсore_data['education'].value_counts() #видим дублирование категорий, при такой выборке смущает низкий процент 


среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [16]:
sсore_data[sсore_data['education'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [17]:
sсore_data['education_id'].value_counts() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

1    15073
0     5202
2      739
3      282
4        6
Name: education_id, dtype: int64

In [18]:
sсore_data[sсore_data['education_id'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [19]:
sсore_data['family_status'].value_counts() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют 
#(возможно, при оценке данных объединить две группы "гражданский брак" и "не женат/не замужем")

женат / замужем          12254
гражданский брак          4139
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

In [20]:
sсore_data[sсore_data['family_status'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [21]:
sсore_data['family_status_id'].value_counts() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

0    12254
1     4139
4     2783
3     1179
2      947
Name: family_status_id, dtype: int64

In [22]:
sсore_data[sсore_data['family_status_id'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [23]:
sсore_data['gender'].value_counts() #в данных выявлена одна строка с ошибочным значением категории "пол". 
#исключим его в силу малого влияния на общее исследование и второстепенность показателя.

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

In [24]:
sсore_data = sсore_data[sсore_data['gender'] != 'XNA']
sсore_data['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

In [25]:
sсore_data[sсore_data['gender'].isnull()].count()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [26]:
sсore_data['income_type'].value_counts() #выявлено оценочное несоответствие групп по возрасту и типу дохода, так, всего 1 студент
#и 3856 пенсионеров, при значительном превосходстве возрастных категорий, куда могут попадать эти группы по доходу. 
#Пример: количество потенциальных студентов (19-21 год) - 176 человек, пенсионеров (старше 60) - 2517. 
#Безусловно, работающие пенсионеры и студенты распространенное явление, поэтому, в реальной жизни у заказчика необходимо уточнять
#способ сбора данных (анкеты,документы, подтвержденные справками с места работы и т.д.)
# эти данные не являются анализируемыми в данном проекте, поэтому, дополнительные исследования по ним не нужны (в ракмах проекта) 

сотрудник          10996
компаньон           5033
пенсионер           3819
госслужащий         1447
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [27]:
sсore_data[sсore_data['income_type'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [28]:
sсore_data['debt'].value_counts() #1724 строка имеет пометку о долге по кредиту

0    19577
1     1724
Name: debt, dtype: int64

In [29]:
sсore_data[sсore_data['debt'].isnull()].count() #при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
dtype: int64

In [30]:
sсore_data['total_income'].value_counts().sort_values(ascending = True)

112874.418757    1
162416.137763    1
122140.327609    1
175780.973093    1
128056.263801    1
                ..
82857.979147     1
80429.374722     1
401469.373495    1
142752.248355    1
150014.128510    1
Name: total_income, Length: 19149, dtype: int64

In [31]:
sсore_data[sсore_data['total_income'].isnull()].count() #вот и наши пропуски
#заполним их

children                2152
days_employed              0
dob_years               2152
education               2152
education_id            2152
family_status           2152
family_status_id        2152
gender                  2152
income_type             2152
debt                    2152
total_income               0
purpose                 2152
update_days_employed    2152
dtype: int64

In [32]:
sсore_data['update_total_income'] = sсore_data['total_income'] 
#вводим новый столбец для заполнения медианным доходом 
#(в нем в цикле будем заполнять пропуски медианными значениями)
for income_type in sсore_data['income_type'].unique(): 
    #находим уникальные значение в категории тип занятости 
    #(медианный доход ищем для каждого типа занятости)
    med_income = sсore_data[sсore_data['income_type'] == income_type]['total_income'].median()
    sсore_data.loc[(sсore_data['update_total_income'].isna()) & (sсore_data['income_type'] == income_type), 'update_total_income'] = med_income
    
    

In [33]:
sсore_data[sсore_data['update_total_income'].isnull()].count()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
update_total_income     0
dtype: int64

In [34]:
sсore_data['purpose'].value_counts() #в колонке с целями кредита идентифицируются неявные дубликаты целей.
#В остальном при верхнеуровневой оценке данных явные ошибки/неточности отсуствуют

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           764
операции с недвижимостью                  670
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     644
операции с жильем                         642
покупка жилья для семьи                   639
жилье                                     636
покупка жилья                             635
недвижимость                              628
операции со своей недвижимостью           626
строительство собственной недвижимости    626
строительство недвижимости                620
строительство жилой недвижимости          619
покупка своего жилья                      618
покупка недвижимости                      615
ремонт жилью                              607
покупка жилой недвижимости                600
на покупку своего автомобиля              501
заняться высшим образованием      

In [35]:
sсore_data[sсore_data['purpose'].isnull()].count()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
update_days_employed    0
update_total_income     0
dtype: int64

### Вывод

В исходных данных обнаружены пропуски в столбцах "общий трудовой стаж в днях" и "ежемесячный доход". Возможные причины - техническая ошибка при выгрузке данных, неправильное заполнение исходных данных, этот параметр не был определен (возможно, доход и стпаж отсуствовали или не был подтвержден). 
Для заполнения пустых строк, необходимо найти медианное значение по каждой категории занятых для столбцов "ежемесячный доход" и "общий трудовой стаж с днях" (среднее значение для таких категорий данных, как доход, не походит, т.к. среднее арифметическое может сильно исказить реальный показатель за счет значительно отклонение минимального и максимального показателя в группе. Для оценки доходов, цены и прочих показателей, нужен средневзвешенный показатель или медиана) и заполнить пропуски этим показателем.
Замена данных произведена через введение нового столбца, в котром пропущенные значения заменены на медианные по группе занятых.
Пропуски в перечисленных столбцах успешно заменены, в остальных столбцах замена пропуском не требуется

### Замена типа данных

In [36]:
#преобразуем тип данных в столбце со средним доходом (update_total_income) в столбец с целочисленными значниями
sсore_data['update_total_income'] = sсore_data['update_total_income'].astype('int')

#преобразуем тип данных в столбце со средним стажем (update_days_employed) в столбец с целочисленными значниями
sсore_data['update_days_employed'] = sсore_data['update_days_employed'].astype('int')

#проверим, что замена проведена корректно

print(sсore_data.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 14 columns):
children                21301 non-null int64
days_employed           19149 non-null float64
dob_years               21301 non-null int64
education               21301 non-null object
education_id            21301 non-null int64
family_status           21301 non-null object
family_status_id        21301 non-null int64
gender                  21301 non-null object
income_type             21301 non-null object
debt                    21301 non-null int64
total_income            19149 non-null float64
purpose                 21301 non-null object
update_days_employed    21301 non-null int64
update_total_income     21301 non-null int64
dtypes: float64(2), int64(7), object(5)
memory usage: 2.4+ MB
None


### Вывод

In [37]:
#Тип данных с столбцах update_total_income и update_days_employed произведен в целочисленный тип данных. В остальных столцах замена не нужна.

### Обработка дубликатов

In [38]:
print(sсore_data[sсore_data.duplicated()].info()) #проверяем наличие полных дубликатов в таблице и считаем их кол-во


<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 2849 to 21415
Data columns (total 14 columns):
children                54 non-null int64
days_employed           0 non-null float64
dob_years               54 non-null int64
education               54 non-null object
education_id            54 non-null int64
family_status           54 non-null object
family_status_id        54 non-null int64
gender                  54 non-null object
income_type             54 non-null object
debt                    54 non-null int64
total_income            0 non-null float64
purpose                 54 non-null object
update_days_employed    54 non-null int64
update_total_income     54 non-null int64
dtypes: float64(2), int64(7), object(5)
memory usage: 6.3+ KB
None


In [39]:
 sсore_data = sсore_data.drop_duplicates().reset_index(drop=True) #удаление дубликатов

In [40]:
print(sсore_data[sсore_data.duplicated()].info()) # проверим эффективность проведенных действий

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 14 columns):
children                0 non-null int64
days_employed           0 non-null float64
dob_years               0 non-null int64
education               0 non-null object
education_id            0 non-null int64
family_status           0 non-null object
family_status_id        0 non-null int64
gender                  0 non-null object
income_type             0 non-null object
debt                    0 non-null int64
total_income            0 non-null float64
purpose                 0 non-null object
update_days_employed    0 non-null int64
update_total_income     0 non-null int64
dtypes: float64(2), int64(7), object(5)
memory usage: 0.0+ bytes
None


In [41]:
# найдем и "удалим" дубликаты в столюце "уровень образования клиента"
# приведем все к единому регистру, добавив новый столбец, и в дальнейшем будем работать с ним.
sсore_data['education_lowercase'] = sсore_data['education'].str.lower()

sсore_data['education_lowercase'].value_counts() #видим дублирование категорий, при такой выборке смущает низкий процент 



среднее                15028
высшее                  5193
неоконченное высшее      738
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

### Вывод

In [42]:
#в исследуемом массиве данных были найдены полные дубликаты (54 строки). Строки для целей проекта были удалены, однако, стоит отметить,
#что полные дубликаты без уникального id строки полными дубликатами считаться не могуту - т.к. вполне вероятен случай полного совпадения
#параметров по двум разным клиентам.

### Лемматизация

In [43]:
#проведем лемматизацию по целям кредита
#создадим новый столбец

from pymystem3 import Mystem
m = Mystem()
from collections import Counter
text = ' '.join(list(sсore_data['purpose'])) #list переводим столбец в список  
lemmas = m.lemmatize(text)

def lemm_text(text):
    return m.lemmatize(text)

sсore_data['purpose_lemmas'] = sсore_data['purpose'].apply(lemm_text)


In [44]:
#новый столбец
sсore_data['purpose_lemmas'].head()

0                 [покупка,  , жилье, \n]
1       [приобретение,  , автомобиль, \n]
2                 [покупка,  , жилье, \n]
3    [дополнительный,  , образование, \n]
4               [сыграть,  , свадьба, \n]
Name: purpose_lemmas, dtype: object

In [45]:
# составим словарь целей

from pymystem3 import Mystem
m = Mystem()
text = ' '.join(list(sсore_data['purpose'])) 
lemmas = m.lemmatize(text)

from collections import Counter
dictionary = Counter(lemmas)
dictionary


Counter({'покупка': 5841,
         ' ': 54516,
         'жилье': 4414,
         'приобретение': 457,
         'автомобиль': 4260,
         'дополнительный': 896,
         'образование': 3971,
         'сыграть': 759,
         'свадьба': 2310,
         'операция': 2576,
         'с': 2886,
         'на': 2202,
         'проведение': 764,
         'для': 1284,
         'семья': 636,
         'недвижимость': 6292,
         'коммерческий': 1299,
         'жилой': 1217,
         'строительство': 1863,
         'собственный': 626,
         'подержать': 838,
         'свой': 2213,
         'со': 623,
         'заниматься': 900,
         'сделка': 933,
         'получение': 1305,
         'высокий': 1359,
         'подержанный': 112,
         'профильный': 431,
         'сдача': 648,
         'ремонт': 602,
         '\n': 1})

In [46]:
#через ручную обработку выделим группы кредитов по целям - выделим самые крупные группы и создадаим словарь целей

dictionary_purpose = ['жилье', 'автомобиль', 'образование', 'свадьба','недвижимость', 'строительство']

#
def purpose_new (purpose_lemmas):
    for pur in dictionary_purpose:
        if pur in purpose_lemmas:
            return pur
        
sсore_data['purpose_new'] = sсore_data['purpose_lemmas'].apply(purpose_new)
sсore_data.groupby('purpose_new').count()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,update_days_employed,update_total_income,education_lowercase,purpose_lemmas
purpose_new,,,,,,,,,,,,,,,,
автомобиль,4260,3851,4260,4260,4260,4260,4260,4260,4260,4260,3851,4260,4260,4260,4260,4260
жилье,4414,3994,4414,4414,4414,4414,4414,4414,4414,4414,3994,4414,4414,4414,4414,4414
недвижимость,6292,5670,6292,6292,6292,6292,6292,6292,6292,6292,5670,6292,6292,6292,6292,6292
образование,3971,3557,3971,3971,3971,3971,3971,3971,3971,3971,3557,3971,3971,3971,3971,3971
свадьба,2310,2077,2310,2310,2310,2310,2310,2310,2310,2310,2077,2310,2310,2310,2310,2310


### Вывод

 #проведена лемматизация целей кредита. Все цели сгруппированы на пять крупных целей.

### Категоризация данных

In [47]:
# проведем категоризацию данных по 4 критериям (исходя из целей проекта): семейное положение, наличие детей и средний доход.
# делим всех заемщиков из выборки на три категории по фактору наличия детей: без детей, 1-2, 3 и более многодетные.

#категоризация по наличию детей.

def chil_number_group(chil):
    if chil == 0:
        return 'без детей'
    if chil <= 2:
        return '1-2 детей'
    return 'многодетные'

sсore_data['children_count'] = sсore_data['children'].apply(chil_number_group)

print(sсore_data.groupby('children_count')['children_count'].count())


children_count
1-2 детей       6832
без детей      14037
многодетные      378
Name: children_count, dtype: int64


In [48]:
#категоризация по семейному положению

def marital_status(status):
    if status == 'женат / замужем':
        return 'состоит в браке'
    return 'не состоит в браке'

sсore_data['marital_status'] = sсore_data['family_status'].apply(marital_status)

print(sсore_data.groupby('marital_status')['marital_status'].count())


marital_status
не состоит в браке     9029
состоит в браке       12218
Name: marital_status, dtype: int64


In [54]:
#категоризация по уровню дохода
#для этого используем метод describe() и полученные с его помощью значения квантилей распредления  
#25%(в выборке 25% процентов имеют такой доход или ниже), по первому квантилю будем определять бедных
#50% (в выборке 50% процентов имеют такой доход или ниже), по второму группу риска
#75% (в выборке 75% имет такой доход или ниже), по третьему группа с устойчивым доходом
# все остальные  "группа в высоким доходом"

income_limits = sсore_data[['update_total_income']].describe().astype('int').T[['25%', '50%', '75%']].values.tolist()


def total_income_status(update_total_income):
    if update_total_income <= income_limits[0][0]:
        return 'бедные'
    if update_total_income <= income_limits[0][1]:
        return 'группа риска'
    if update_total_income < income_limits[0][2]:
        return 'группа с устойчивым доходом'
    return 'богатые'

sсore_data['total_income_status'] = sсore_data['update_total_income'].apply(total_income_status)
sсore_data['total_income_status'].value_counts()

# получаем наши квантили и методом Т транспонируем для подстановки в будущую формулу (при измнении данных поможет избежать
#ручной подстановки)
# формируем из них список




группа риска                   5433
богатые                        5313
бедные                         5312
группа с устойчивым доходом    5189
Name: total_income_status, dtype: int64

### Вывод 


#в исследуемом массиве данных были найдены следующие закономерности:
#люди, не имеющие детей вдвое чаще берут кредиты,чем люди, не имеющие детей, а многодетные семьи и вовсе избегают кредитов 
#(менее 2%)
#люди, не состоящие в браке крайне редко берут кредиты, относительно людей, состоящих в зарегестрированном браке
#(соотношение 1:55). Проведена категоризация данных по доходу на 4 группы: бедные,группа риска,группа с устойчивым доходом и #богатые. Большая часть - "группа риска", впрочем, совсем незначительно превышает остальные группы. Можно сказать о примерно #равномерном распределении групп


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [67]:
sсore_data_pivot = sсore_data.pivot_table(index='children_count', columns='debt', values='gender', aggfunc='count')

sсore_data_pivot['total_debt'] = sсore_data_pivot[0] + sсore_data_pivot[1]

sсore_data_pivot['percent_non_repayment'] = sсore_data_pivot[1] / sсore_data_pivot['total_debt']


print(sсore_data_pivot.sort_values( by = 'percent_non_repayment', ascending=False))


debt                0     1  total_debt  percent_non_repayment
children_count                                                
1-2 детей        6197   635        6832               0.092945
многодетные       347    31         378               0.082011
без детей       12979  1058       14037               0.075372


### Вывод

#согласно полученным данным, наибольший процент невозврата кредитов в срок в категории заемщики, имеющие 1-2 ребенка (9%). #далее следуют многодетные заемщики (8%) и бездетные (7.5%). Прямой зависимости от количества детей не наблюдается. Т.к., в #случае прямой зависимости, рейтинг строился бы по убыванию или возрастанию многодетные, 1-2 ребенка, без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [68]:
sсore_data_pivot_1 = sсore_data.pivot_table(index='marital_status', columns='debt', values='gender', aggfunc='count')

sсore_data_pivot_1['total_debt'] = sсore_data_pivot_1[0] + sсore_data_pivot_1[1]

sсore_data_pivot_1['percent_non_repayment'] = sсore_data_pivot_1[1] / sсore_data_pivot_1['total_debt']


print(sсore_data_pivot_1.sort_values( by = 'percent_non_repayment', ascending=False))

debt                    0    1  total_debt  percent_non_repayment
marital_status                                                   
не состоит в браке   8228  801        9029               0.088714
состоит в браке     11295  923       12218               0.075544


### Вывод

#Группа заемщиков, состоящая в браке реже возвращает кредиты с просчрочкой, чем группа не состоящих в браке. Вероятность #возрата в срок выше у группы заемщиков, состоящих в браке.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [69]:
sсore_data_pivot_2 = sсore_data.pivot_table(index='total_income_status', columns='debt', values='gender', aggfunc='count')

sсore_data_pivot_2['total_debt'] = sсore_data_pivot_2[0] + sсore_data_pivot_2[1]

sсore_data_pivot_2['percent_non_repayment'] = sсore_data_pivot_2[1] / sсore_data_pivot_2['total_debt']


print(sсore_data_pivot_2.sort_values( by = 'percent_non_repayment', ascending=False))
                                        

                                            

debt                            0    1  total_debt  percent_non_repayment
total_income_status                                                      
группа риска                 4955  478        5433               0.087981
группа с устойчивым доходом  4748  441        5189               0.084987
бедные                       4887  425        5312               0.080008
богатые                      4933  380        5313               0.071523


### Вывод

#чаще всего кредиты невозвращают заещмики из группы риска,что прямо противоречит гипотизе об имеющейся зависимости от уровня #дохода. В случае ее подтверждения, самыми неблагонадежными являлись бы бедные, далее, группа риска, группа с устойчивым #доходом и богатые
#cамый добросовестные плательщики - богатные люди, и бедные.
#зависимости вовзрата кредита в срок и уровнем дохода не выявлено

- Как разные цели кредита влияют на его возврат в срок?

In [70]:
sсore_data_pivot_3 = sсore_data.pivot_table(index='purpose_new', columns='debt', values='gender', aggfunc='count')

sсore_data_pivot_3['total_debt'] = sсore_data_pivot_3[0] + sсore_data_pivot_3[1]

sсore_data_pivot_3['percent_non_repayment'] = sсore_data_pivot_3[1] / sсore_data_pivot_3['total_debt']


print(sсore_data_pivot_3.sort_values( by = 'percent_non_repayment', ascending=False))

debt             0    1  total_debt  percent_non_repayment
purpose_new                                               
автомобиль    3863  397        4260               0.093192
образование   3602  369        3971               0.092924
свадьба       2129  181        2310               0.078355
недвижимость  5821  471        6292               0.074857
жилье         4108  306        4414               0.069325


### Вывод

видим интересную картину по целям кредита.
При покупке движимого имущества (автомобиль) самый высокий процент невозврата в срок. 
Самый низкий покупка жилья - что косвенно подтверждает результаты по возврату кредита по группе состоящих в браке (сделаем допущение, что кредит берется под покупку семейного жилья, после заключения брака)



<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

Были исследованы статистические данные по возвратам кредитов в срок.
Основной целью исследования было выявление закономерности:влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Выводы: количество детей прямо не влияет, семеное положение влияет (возвраткредита с просрочкой по заемщикам, состоящим в браке, ниже).
По целям кредита выявлена самая стабильная группа по вовзрату кредита в срок - это покупка жилья, самая нестабильная - автокредит.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)
